---
# Objetivo: Treinar modelo BERT no conjunto de dados filtrado pela categoria Fruta. Variavel target o tema dessa categoria. Ex. Banana, Maça, Uva
---

**1- Descrição do problema ou tarefa:**

Prever o tema da categoria Frutas a partir das perguntas.

**2- Descrição da solução de IA:**
Treinamento supervisionado de modelo de classificação os temas da categoria Fruta (15 classes) com os dados de 7035 perguntas.

**3- Fonte de dados:**


Livro de perguntas e respostas da Embrapa
https://mais500p500r.sct.embrapa.br/view/index.php4

**4-Variáveis independentes:** 
perguntas

**5- Variável dependente:** 
Temas da cagetoria frutas:Banana, Maça, Uva...

-----------------------------------------------------------------
Autor do caderno: Wellington Rangel
Data: 23/11/2021

In [ ]:
# install ktrain on Google Colab
!pip3 install ktrain

     |████████████████████████████████| 25.3 MB 28.8 MB/s 
     |████████████████████████████████| 6.8 MB 19.3 MB/s 
     |████████████████████████████████| 981 kB 45.6 MB/s 
     |████████████████████████████████| 263 kB 50.4 MB/s 
     |████████████████████████████████| 2.8 MB 20.1 MB/s 
     |████████████████████████████████| 1.2 MB 38.5 MB/s 
     |████████████████████████████████| 468 kB 48.9 MB/s 
     |████████████████████████████████| 596 kB 45.4 MB/s 
     |████████████████████████████████| 895 kB 38.4 MB/s 
     |████████████████████████████████| 61 kB 490 kB/s 
     |████████████████████████████████| 3.3 MB 36.7 MB/s 
  Created wheel for ktrain: filename=ktrain-0.28.3-py3-none-any.whl size=25292659 sha256=41bf23076d8b117b7c8b2b95d804a1c0989efb438871f41a6aa4b3943613269f
  Stored in directory: /root/.cache/pip/wheels/6a/7e/c3/f46cdfc2b81c54424923b1405d7e670c35cacc11ada9a47b1c
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=a756f032a9d9a38

In [ ]:
# import ktrain and the ktrain.text modules
import ktrain
from ktrain import text

In [ ]:
ktrain.__version__

'0.28.3'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/projeto/

/content/drive/MyDrive/Colab Notebooks/projeto


In [ ]:
import pandas as pd
df = pd.read_csv('dados_agrupados.csv',  sep=',', low_memory=False,encoding='latin-1' )


In [ ]:
df

,Numero,Livro,Capitulo,Pergunta,Resposta,Target,Target_final,Target_final1,Target_final2,Target_final3
0,1,Pera,Generalidades,Qual é o centro de origem da pereira?,São citados três centros de origem da pereira:...,Pera,Frutas,NaN,Frutas,NaN
1,2,Pera,Generalidades,Qual é o centro de origem mais importante?,O centro do Oriente Médio é considerado de imp...,Pera,Frutas,NaN,Frutas,NaN
2,3,Pera,Generalidades,Como ocorreu a disseminação da pereira pelo mu...,"Com base em estudos bioquímicos, verificou-se ...",Pera,Frutas,NaN,Frutas,NaN
3,4,Pera,Generalidades,Quais são as espécies de pereira mais cultivad...,"Na Europa, na América do Norte, na América do ...",Pera,Frutas,NaN,Frutas,NaN
4,5,Pera,Generalidades,Quando a pereira foi introduzida no Brasil?,Não há relatos na literatura sobre a introduçã...,Pera,Frutas,NaN,Frutas,NaN
...,...,...,...,...,...,...,...,...,...,...
16591,497,Uva,NaN,Qual é a produção nacional e a mundial de vinhos?,O Brasil produz cerca de 300 milhões de litros...,Uva,Frutas,NaN,Frutas,NaN
16592,498,Uva,NaN,Quais são os maiores produtores e os maiores e...,"Os maiores produtores são a França, a Espanha ...",Uva,Frutas,NaN,Frutas,NaN
16593,499,Uva,NaN,Que quantidade de vinho o Brasil importa anual...,"Mais de 55 milhões de litros de vinho fino, o ...",Uva,Frutas,NaN,Frutas,NaN
16594,500,Uva,NaN,Qual é o consumo per capita dos países que mai...,"França e Luxemburgo, 55 litros; Portugal, 48 l...",Uva,Frutas,NaN,Frutas,NaN


In [ ]:
df = df.loc[df['Target_final'] == 'Frutas']

In [ ]:

df = df[['Pergunta', 'Target']]


In [ ]:
df.head()

,Pergunta,Target
0,Qual é o centro de origem da pereira?,Pera
1,Qual é o centro de origem mais importante?,Pera
2,Como ocorreu a disseminação da pereira pelo mu...,Pera
3,Quais são as espécies de pereira mais cultivad...,Pera
4,Quando a pereira foi introduzida no Brasil?,Pera


In [ ]:
df.tail()

,Pergunta,Target
16591,Qual é a produção nacional e a mundial de vinhos?,Uva
16592,Quais são os maiores produtores e os maiores e...,Uva
16593,Que quantidade de vinho o Brasil importa anual...,Uva
16594,Qual é o consumo per capita dos países que mai...,Uva
16595,Mais alguma pergunta?,Uva


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')


In [ ]:
sub = df[[ 'Pergunta', 'Target']]

In [ ]:
sub['texto_sem_stopwords'] = sub['Pergunta'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [ ]:
sub = sub.dropna()

In [ ]:
sub

,Pergunta,Target,texto_sem_stopwords
0,Qual é o centro de origem da pereira?,Pera,Qual centro origem pereira?
1,Qual é o centro de origem mais importante?,Pera,Qual centro origem importante?
2,Como ocorreu a disseminação da pereira pelo mu...,Pera,Como ocorreu disseminação pereira mundo?
3,Quais são as espécies de pereira mais cultivad...,Pera,Quais espécies pereira cultivadas mundo?
4,Quando a pereira foi introduzida no Brasil?,Pera,Quando pereira introduzida Brasil?
...,...,...,...
16591,Qual é a produção nacional e a mundial de vinhos?,Uva,Qual produção nacional mundial vinhos?
16592,Quais são os maiores produtores e os maiores e...,Uva,Quais maiores produtores maiores exportadores ...
16593,Que quantidade de vinho o Brasil importa anual...,Uva,Que quantidade vinho Brasil importa anualmente?
16594,Qual é o consumo per capita dos países que mai...,Uva,Qual consumo per capita países consomem vinhos?


In [ ]:
# Separa os dados de treino e teste (proporção 80/20, stratify para classes desbalanceadas)
from sklearn.model_selection import train_test_split
m_X_train, m_X_test, m_y_train, m_y_test = train_test_split(df.Pergunta, df.Target, test_size=0.2, 
                                                    stratify=df.Target,random_state=42)




In [ ]:
type(m_X_train)

pandas.core.series.Series

In [ ]:
m_y_train

14582             Pequenas Frutas
5162                         Caju
1116     Pessego-Nectarina-Ameixa
433                          Pera
1355     Pessego-Nectarina-Ameixa
                   ...           
2999                      Algodão
14579             Pequenas Frutas
11360                       Manga
10326                       Mamão
11553                       Manga
Name: Target, Length: 5628, dtype: object

In [ ]:
x_train = m_X_train.tolist()
y_train = m_y_train.tolist()
x_test = m_X_test.tolist()
y_test = m_y_test.tolist()

In [ ]:
type(x_train)

list

In [ ]:

type(x_train)


list

In [ ]:
classes = df.Target.unique().tolist()

In [ ]:
type(classes)

list

In [ ]:
classes

['Pera',
 'Pessego-Nectarina-Ameixa',
 'Abacaxi',
 'Algodão',
 'Banana',
 'Caju',
 'Citros',
 'Coco',
 'Mamão',
 'Manga',
 'Maçã',
 'Maracujá',
 'Pequenas Frutas',
 'Uva']

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=classes,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

downloading pretrained BERT model (multi_cased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: pt


Is Multi-Label? False
preprocessing test...
language: pt


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:621: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


task: text classification


## Treinando o modelo

In [ ]:
# you can disregard the deprecation warnings arising from using Keras 2.2.4 with TensorFlow 1.14.
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=8)

Is Multi-Label? False
maxlen is 350
done.


In [ ]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
704/704 [==============================] - 967s 1s/step - loss: 2.0319 - accuracy: 0.3491
Epoch 2/4
704/704 [==============================] - 942s 1s/step - loss: 1.1644 - accuracy: 0.6347
Epoch 3/4
704/704 [==============================] - 943s 1s/step - loss: 0.8490 - accuracy: 0.7265
Epoch 4/4
704/704 [==============================] - 943s 1s/step - loss: 0.4949 - accuracy: 0.8399


In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=classes)

                          precision    recall  f1-score   support

                    Pera       0.66      0.73      0.69       100
Pessego-Nectarina-Ameixa       0.73      0.78      0.75       100
                 Abacaxi       0.74      0.71      0.72       100
                 Algodão       0.79      0.74      0.76       102
                  Banana       0.65      0.55      0.59       100
                    Caju       0.77      0.71      0.74       101
                  Citros       0.67      0.71      0.69       100
                    Coco       0.59      0.67      0.63       100
                   Mamão       0.84      0.75      0.79       100
                   Manga       0.63      0.66      0.64       101
                    Maçã       0.75      0.75      0.75       101
                Maracujá       0.61      0.51      0.56       101
         Pequenas Frutas       0.68      0.78      0.73       101
                     Uva       0.72      0.73      0.73       100

        

array([[73,  1,  1,  0,  5,  3,  3,  3,  0,  0,  3,  1,  3,  4],
       [ 3, 78,  0,  0,  2,  3,  3,  2,  1,  2,  2,  1,  3,  0],
       [ 2,  2, 71,  0,  0,  1,  8,  4,  4,  1,  4,  0,  1,  2],
       [ 6,  5,  1, 75,  2,  2,  2,  1,  1,  1,  0,  1,  2,  3],
       [ 3,  2,  6,  0, 55,  0, 10, 14,  1,  3,  3,  1,  2,  0],
       [ 6,  4,  2,  2,  0, 72,  0,  3,  0,  1,  0,  2,  3,  6],
       [ 6,  4,  4,  0,  4,  2, 71,  3,  2,  0,  0,  1,  2,  1],
       [ 0,  3,  3,  4,  8,  2,  2, 67,  0,  1,  4,  3,  2,  1],
       [ 1,  1,  3,  4,  2,  1,  1,  3, 75,  1,  2,  2,  4,  0],
       [ 0,  2,  2,  3,  1,  1,  0,  2,  2, 67,  2,  9,  4,  6],
       [ 3,  0,  2,  1,  3,  1,  1,  2,  2,  3, 76,  4,  1,  2],
       [ 5,  2,  1,  0,  2,  1,  2,  2,  0, 20,  6, 52,  7,  1],
       [ 2,  2,  0,  0,  0,  0,  2,  2,  1,  3,  0,  8, 79,  2],
       [ 1,  1,  0,  6,  1,  5,  1,  5,  0,  4,  0,  0,  3, 73]])

## Usando o modelo treinado


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.get_classes()

['Abacaxi',
 'Algodão',
 'Banana',
 'Caju',
 'Citros',
 'Coco',
 'Mamão',
 'Manga',
 'Maracujá',
 'Maçã',
 'Pequenas Frutas',
 'Pera',
 'Pessego-Nectarina-Ameixa',
 'Uva']

In [ ]:
df[0:1]

,Pergunta,Target
0,Qual é o centro de origem da pereira?,Pera


In [ ]:
predictor.predict(df[0:1]['Pergunta'].to_string())

'Pera'

In [ ]:
df[0:1]['Pergunta']

0    Qual é o centro de origem da pereira?
Name: Pergunta, dtype: object

In [ ]:
# we can visually verify that our prediction of 'sci.med' for this document is correct
print(df[0:1]['Target_final'])

0    Frutas
Name: Target_final, dtype: object


In [ ]:
predictor.save('/content/drive/MyDrive/Colab Notebooks/projeto/modelos/my_predictor_frutas')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
